In [1]:
!pip install tensorflow

In [2]:
!pip install keras

In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras import layers

**ĐỊNH NGHĨA LỚP TÙY CHỈNH (CUSTOM LAYER)**

In [4]:
@tf.keras.utils.register_keras_serializable()
class AttentionLayer(layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Khởi tạo các tham số trọng số (Trainable Parameters), Đây là nơi các tham số được khởi tạo ngẫu nhiên ban đầu (glorot_uniform/zeros)
        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1]),
                                initializer='glorot_uniform', trainable=True, name='W_attention')
        self.b = self.add_weight(shape=(input_shape[-1],),
                                initializer='zeros', trainable=True, name='b_attention')
        self.u = self.add_weight(shape=(input_shape[-1],1),
                                initializer='glorot_uniform', trainable=True, name='u_context')
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        # inputs: (batch, timesteps, units)
        v = tf.tanh(tf.tensordot(inputs, self.W, axes=1) + self.b)
        vu = tf.tensordot(v, self.u, axes=1)
        alphas = tf.nn.softmax(vu, axis=1)
        output = tf.reduce_sum(inputs * alphas, axis=1)
        return output

    def get_config(self):
        config = super().get_config()
        return config

**Hàm phân tích model**

In [5]:
def inspect_saved_model(filepath):
    print(f" Đang tải mô hình từ: {filepath} ...")

    try:
        # Load model với custom_objects
        model = load_model(filepath, custom_objects={'AttentionLayer': AttentionLayer})
        print(" Tải mô hình thành công!\n")
    except Exception as e:
        print(f" Lỗi tải mô hình: {e}")
        return None

    # --- A. CẤU TRÚC TỔNG QUAN ---
    print("="*40)
    print(" A. CẤU TRÚC TỔNG QUAN (SUMMARY)")
    print("="*40)
    model.summary()
    print("\n")

    # --- B. CHI TIẾT HYPERPARAMETERS (CẤU HÌNH) ---
    print("="*40)
    print(" B. CHI TIẾT CẤU HÌNH LAYER (HYPERPARAMETERS)")
    print("="*40)
    for i, layer in enumerate(model.layers):
        conf = layer.get_config()
        print(f" Layer {i}: {layer.name} ({layer.__class__.__name__})")

        # In các thông số cấu hình quan trọng
        if 'units' in conf: print(f"   - Units (Nơ-ron): {conf['units']}")
        if 'activation' in conf: print(f"   - Activation: {conf['activation']}")
        if 'rate' in conf: print(f"   - Dropout Rate: {conf['rate']}")
        if 'dropout' in conf: print(f"   - LSTM Dropout: {conf['dropout']}")
        if 'recurrent_dropout' in conf: print(f"   - Recurrent Dropout: {conf['recurrent_dropout']}")
        if 'return_sequences' in conf: print(f"   - Return Sequences: {conf['return_sequences']}")
        print("-" * 20)
    print("\n")

    # --- C. CHI TIẾT TRỌNG SỐ (WEIGHTS & BIASES) ---
    print("="*40)
    print(" C. THÔNG SỐ TRỌNG SỐ ĐÃ TRAIN (WEIGHTS)")
    print(" (Đây là các giá trị thực tế sau khi train, không còn là ngẫu nhiên nữa)")
    print("="*40)

    total_params = 0
    for layer in model.layers:
        weights = layer.get_weights()
        if len(weights) > 0:
            print(f" Layer: {layer.name}")
            for idx, w in enumerate(weights):
                print(f"   - Param {idx} Shape: {w.shape}")
                print(f"     Min: {np.min(w):.4f} | Max: {np.max(w):.4f} | Mean: {np.mean(w):.4f}")
                print(f"     Values (5 đầu tiên): {w.flatten()[:5]}")
            total_params += layer.count_params()
            print("-" * 20)

    # --- D. PHÂN TÍCH ĐẦU VÀO (CHO HÀM CREATE_SEQUENCES) ---
    print("\n")
    print("="*40)
    print(" D. THÔNG SỐ ĐẦU VÀO (INPUT SHAPE)")
    print("="*40)

    input_shapes = [inp.shape for inp in model.inputs]
    print(f"Số lượng đầu vào: {len(input_shapes)}")

    timesteps = None
    seq_feats = None
    dense_feats = None

    if len(input_shapes) >= 1:
        # Giả định input đầu tiên là Sequence (Batch, Timesteps, Features)
        shape_1 = input_shapes[0]
        timesteps = shape_1[1]
        seq_feats = shape_1[2]
        print(f" Input 1 (Sequence): Timesteps = {timesteps}, Features = {seq_feats}")

    if len(input_shapes) >= 2:
        # Giả định input thứ hai là Dense/Static (Batch, Features)
        shape_2 = input_shapes[1]
        dense_feats = shape_2[1]
        print(f" Input 2 (Dense): Features = {dense_feats}")

    print(f"\n KẾT LUẬN CHO DATA PROCESSING:")
    print(f"   Bạn cần cắt dữ liệu với cửa sổ trượt (window size) = {timesteps}")
    print(f"   Bạn cần chọn đúng {seq_feats} cột cho dữ liệu chuỗi.")
    if dense_feats:
        print(f"   Bạn cần chọn đúng {dense_feats} cột cho dữ liệu tĩnh.")

    return model

**Chạy script**

In [6]:
filename = '/content/drive/MyDrive/Colab Notebooks/Best_model_LSTM.keras'
model = inspect_saved_model(filename)

 Đang tải mô hình từ: /content/drive/MyDrive/Colab Notebooks/Best_model_LSTM.keras ...
 Tải mô hình thành công!

 A. CẤU TRÚC TỔNG QUAN (SUMMARY)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ seq_input (InputLayer)          │ (None, 120, 11)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 120, 256)       │       143,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 120, 128)       │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 120, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_layer                 │ (None, 128)            │        16,640 │
│ (AttentionLayer)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,016,645 (3.88 MB)

 Trainable params: 338,881 (1.29 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 677,764 (2.59 MB)



 B. CHI TIẾT CẤU HÌNH LAYER (HYPERPARAMETERS)
 Layer 0: seq_input (InputLayer)
--------------------
 Layer 1: bidirectional (Bidirectional)
--------------------
 Layer 2: bidirectional_1 (Bidirectional)
--------------------
 Layer 3: dropout (Dropout)
   - Dropout Rate: 0.2
--------------------
 Layer 4: attention_layer (AttentionLayer)
--------------------
 Layer 5: dense (Dense)
   - Units (Nơ-ron): 64
   - Activation: relu
--------------------
 Layer 6: dense_1 (Dense)
   - Units (Nơ-ron): 64
   - Activation: relu
--------------------
 Layer 7: dropout_1 (Dropout)
   - Dropout Rate: 0.15
--------------------
 Layer 8: dense_2 (Dense)
   - Units (Nơ-ron): 32
   - Activation: relu
--------------------
 Layer 9: dense_3 (Dense)
   - Units (Nơ-ron): 1
   - Activation: linear
--------------------


 C. THÔNG SỐ TRỌNG SỐ ĐÃ TRAIN (WEIGHTS)
 (Đây là các giá trị thực tế sau khi train, không còn là ngẫu nhiên nữa)
 Layer: bidirectional
   - Param 0 Shape: (11, 512)
     Min: -0.3581 | Max: